<a href="https://colab.research.google.com/github/21cs076/Detection-of-Landslides/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics==8.3.72

In [ ]:
from ultralytics import YOLO
from google.colab import files

model = YOLO("yolo11s.pt")
model.train(
    data = "/content/drive/MyDrive/Landslide_Dataset/data.yaml",  # Path to your data.yaml file
    epochs=80,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory), (16), 32, 64, 128
    imgsz=640,  # Image size for training ((640 is a good choice), 736)
    project = "/content/drive/MyDrive/Landslide_Dataset",  # Directory to save training results
    name="runs",  # Name for the training run
    exist_ok=True  # Overwrite existing results if any
    save_period=10         # Save the model every 10 epochs
)
files.download("/content/drive/MyDrive/Landslide_Dataset/runs/weights/best.pt")

In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/Landslide_Dataset/runs/weights/best.pt")
#model = YOLO("/content/drive/MyDrive/models/lbest70m.pt")

tuned_conf = 0.6  # Increase confidence threshold to reduce false positives
tuned_iou  = 0.5  # Adjust NMS IoU threshold to control box suppression behavior

results = model.predict(
    source="/content/drive/MyDrive/Landslide_Dataset/test/images",  # Path to test images
    conf=tuned_conf,  # Apply tuned confidence threshold
    iou=tuned_iou,    # Apply tuned NMS IoU threshold
    show=True
)

# Alternatively, you can also pass these parameters when calling model.val.
# Note: Depending on your version, model.val may or may not directly support conf and iou parameters.
metrics = model.val(
    data="/content/drive/MyDrive/Landslide_Dataset/data.yaml",
    split='test',
    conf=tuned_conf,  # tuned confidence threshold during evaluation
    iou=tuned_iou     # tuned IoU threshold during evaluation
)

print(f"mAP50: {metrics.box.map50}")  # Average Precision at IoU 0.5
print(f"mAP75: {metrics.box.map75}")  # Average Precision at IoU 0.75

In [ ]:


metrics = model.val(data="/content/drive/MyDrive/Landslide_Dataset/data.yaml", split='test')
print(f"mAP50: {metrics.box.map50}") # mAP refers to the Average Precision at a single IoU threshold, often 50%.
print(f"mAP75: {metrics.box.map75}") # mAP at IoU threshold of 75% (mAP75)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 21.2MB/s]
val: Scanning /content/drive/MyDrive/Landslide_Dataset/test/labels... 574 images, 44 backgrounds, 0 corrupt: 100%|██████████| 574/574 [04:16<00:00,  2.24it/s]


val: New cache created: /content/drive/MyDrive/Landslide_Dataset/test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [04:49<00:00,  8.04s/it]


                   all        574       1045      0.664      0.593      0.648       0.33
               Fissure        177        338      0.716      0.343      0.474      0.202
             Landslide        426        707      0.612      0.843      0.821      0.458
Speed: 4.6ms preprocess, 431.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val
mAP50: 0.647900577669211
mAP75: 0.2795648106204136


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
